In [366]:
import pandas as pd
import re
import os
os.chdir('/Users/D-Money/Desktop/Python')

In [367]:
param = pd.read_excel('Parameters.xlsx', sheet_name = 'Params')
param

,Table,Where
0,FactSalesDocHeader,"WHERE A in (1, 2) AND B is not NULL AND D is n..."
1,FactSalesDocItem,WHERE SalesCode = 1


In [368]:
a = param.iloc[0, 1]
a

'WHERE A in (1, 2) AND B is not NULL AND D is not nUll and D in (1, 7)'

In [369]:
df = pd.read_excel('Parameters.xlsx', sheet_name = 'Data')
df

,A,B,C,D
0,1,a,2,1.0
1,1,b,4,2.0
2,1,a,6,NaN
3,2,b,8,7.0
4,2,a,10,7.0


In [370]:
def convert_to_pandas(d, w):

    # Remove WHERE
    w = w[6:]
    
    # Uppercase AND, IN, NOT, ...
    w = re.compile(re.escape(' AND '), re.IGNORECASE).sub(' AND ', w)
    w = re.compile(re.escape(' IN '), re.IGNORECASE).sub(' IN ', w)
    w = re.compile(re.escape(' NOT '), re.IGNORECASE).sub(' NOT ', w)
    w = re.compile(re.escape(' IS '), re.IGNORECASE).sub(' IS ', w)
    w = re.compile(re.escape(' NULL '), re.IGNORECASE).sub(' NULL ', w)
    w = re.compile(re.escape(' IS NULL'), re.IGNORECASE).sub(' IS NULL', w)
    w = re.compile(re.escape(' IS NOT NULL'), re.IGNORECASE).sub(' IS NOT NULL', w)
        
    # Split into separete statements
    w = [x.strip() for x in w.split(' AND ')]

    # Fix the = filters
    w = [x.replace(' = ', ' == ') for x in w]
        
    # Fix the IN filters
    w = [x.replace(' IN ', '.isin') for x in w]
    
    # Fix the NOT IN filters
    w = ['~' + x if 'NOT.isin' in x else x for x in w]
    w = [x.replace('NOT.isin', '.isin', 1) for x in w]
    
    # Fix the IS NULL filters
    w = [x.replace(' IS NULL ', '.isnull()') for x in w]
    w = [x.replace(' IS NULL', '.isnull()') for x in w]

    
    # Fix the IS NOT NULL filters
    w = ['~' + x.replace(' IS NOT NULL ', '.isnull()') if 'IS NOT NULL ' in x else x for x in w]
    w = ['~' + x.replace(' IS NOT NULL', '.isnull()') if 'IS NOT NULL' in x else x for x in w]


    
    # Turn into a filter statement
    w = [x.replace('(', '([', 1).replace(')', '])', 1) if '.isin' in x else x for x in w]
    out = []
    for x in w:
        if '~' in x:
            out.append('~' + d + "['" + x[1] + "']" + x[2:])
        else:
            out.append(d + "['" + x[0] + "']" + x[1:])
    
    out = ['(' + x + ')' for x in out]

    # Join into a single string
    out = ' & '.join(out)
    out = d + '[' + out + ']'
    
    # last fix
    out = out.replace('] .isin', '].isin')
    
    return(out)

In [371]:
print(convert_to_pandas('df', a))

df[(df['A'].isin([1, 2])) & (~df['B'].isnull()) & (~df['D'].isnull()) & (df['D'].isin([1, 7]))]


In [372]:
exec('filter_df = ' + convert_to_pandas('df', a))

In [373]:
filter_df

,A,B,C,D
0,1,a,2,1.0
3,2,b,8,7.0
4,2,a,10,7.0


In [374]:
df

,A,B,C,D
0,1,a,2,1.0
1,1,b,4,2.0
2,1,a,6,NaN
3,2,b,8,7.0
4,2,a,10,7.0
